In [1]:
# beartype ensures that the code is type-checked at runtime
%load_ext ipython_beartype
%beartype

%load_ext autoreload
%autoreload 2

# Project Goal

![Motion-retargeting overview](../../media/motion-retargeting.png)


Build an easy to setup end-to-end pipeline that converts **time-synced multi-camera footage** into **axis–angle joint angles + metric 3-D positions** of every finger joint.  
This dataset becomes plug-and-play “motion fuel” for:

* **Robotic retargeting** – Drive anthropomorphic or multi-finger grippers directly from human demonstrations.  
* **Learning fine motor skills** – High-resolution, constraint-aware labels let models capture subtle grasps, in-hand manipulation, and tactile exploration—movements that raw meshes or XYZ clouds alone fail to encode.

In short, precise joint angles are the missing link between human dexterity and robot hands.

### Why do we need a kinematic skeleton with joint angles?

- **Rig-agnostic pose** – Captures only relative bone rotations, so the same grasp can be transfered to any avatar or robot hand.  
- **Easy retargeting** – A single FK pass transfers motion to MANO, MediaPipe, game rigs, or grippers.  
- **Tiny storage** – 20 joints × 3 floats ≈ 60 numbers per frame—orders of magnitude smaller than meshes.  
- **Built-in constraints** – Bone lengths, joint limits, and couplings are trivial to enforce in angle space.  
- **Gradient-friendly** – FK is differentiable, enabling fast optimisation from 2-D reprojection errors.  
- **Sensor fusion-ready** – IMUs, mocap markers, or language commands map cleanly into the same angles.

### Capture-to-Angles Pipeline  
**Input :** time-synced, calibrated cameras (intrinsics + extrinsics)  
**Output :** axis–angle joint angles **θ** + 3-D joint positions in meters

| Step | Input | Process | Output |
|------|-------|---------|--------|
| 1 | RGB frames | 2-D keypoint detector | Pixel joints |
| 2 | Pixel joints + cams | Triangulation / PnP | 3-D joints (m) |
| 3 | 3-D joints | Inverse kinematics | Axis-angle **θ** |
| 4 | **θ** | FK sanity check | Rebuilt 3-D joints |
| 5 | **θ**, 3-D joints | Serialize | **θ** + xyz (m) |

In [2]:
# Add these lines to the cell with id 34bb5824
import warnings

import jax.numpy as jnp
import numpy as np
import rerun as rr
from jaxtyping import Array, Float
from simplecv.data.exoego.skeleton.mediapipe import MEDIAPIPE_ID2NAME, MEDIAPIPE_IDS, MEDIAPIPE_LINKS

from pi0_lerobot.mano.mano_optimization_jax import HandSide
from pi0_lerobot.skeletons.coco_133 import COCO_133_ID2NAME, COCO_133_IDS, COCO_133_LINKS

warnings.filterwarnings('ignore', category=RuntimeWarning)


In [3]:

# Helps provide context for the rerun viewer, to visualize the difference between triangulated and optimized hand joints
def set_pose_annotation_context() -> None:
    rr.log(
        "/",
        rr.AnnotationContext(
            [
                rr.ClassDescription(
                    info=rr.AnnotationInfo(id=0, label="Triangulated Hand", color=(0, 0, 255)),
                    keypoint_annotations=[
                        rr.AnnotationInfo(id=id, label=name) for id, name in MEDIAPIPE_ID2NAME.items()
                    ],
                    keypoint_connections=MEDIAPIPE_LINKS,
                ),
                rr.ClassDescription(
                    info=rr.AnnotationInfo(id=1, label="Optimized Hand", color=(255, 0, 0)),
                    keypoint_annotations=[
                        rr.AnnotationInfo(id=id, label=name) for id, name in MEDIAPIPE_ID2NAME.items()
                    ],
                    keypoint_connections=MEDIAPIPE_LINKS,
                ),
                rr.ClassDescription(
                    info=rr.AnnotationInfo(id=2, label="Wholebody 133", color=(0, 0, 0)),
                    keypoint_annotations=[
                        rr.AnnotationInfo(id=id, label=name) for id, name in COCO_133_ID2NAME.items()
                    ],
                    keypoint_connections=COCO_133_LINKS,
                ),
            
            ]
        ),
        static=True,
    )

### Example dataset – HO-Cap

We’ll use the [HO-Cap](https://irvlutd.github.io/HOCap/) dataset, which ships with:

- ⏱ Time-synchronized RGB videos  
- 📷 Camera intrinsics **K** and extrinsics **R | t**  
- 🎯 2-D + 3-D hand keypoints (pixels / meters)  
- ✋ MANO parameters (shape β, joint angles θ)

In [4]:
from pathlib import Path

from jaxtyping import Float32, UInt8
from numpy import ndarray
from rerun.blueprint import Blueprint
from simplecv.apis.view_exoego_data import create_blueprint
from simplecv.camera_parameters import PinholeParameters
from simplecv.data.exoego.base_exo_ego import ExoData
from simplecv.data.exoego.hocap import HOCapSequence
from simplecv.rerun_log_utils import log_pinhole

data_path = Path("../../data/hocap/sample")
assert data_path.exists(), f"Data path {data_path} does not exist, please check the path."
sequence: HOCapSequence = HOCapSequence(
    data_path=data_path,
    sequence_name="20231024_180733",
    subject_id="8",
    load_labels=True,
)

parent_log_path: Path = Path("world")

exo_cam_log_paths: list[Path] = [parent_log_path / exo_cam.name for exo_cam in sequence.exo_cam_list]
exo_video_log_paths: list[Path] = [cam_log_paths / "pinhole" / "video" for cam_log_paths in exo_cam_log_paths]

blueprint: Blueprint = create_blueprint(exo_video_log_paths=exo_video_log_paths, num_videos_to_log=8)

Found HoloLens camera: hololens_kv5h72


Indexing depth images: 100%|██████████| 8/8 [00:00<00:00, 450.04it/s]


## Visualize the first 50 frames of the dataset

In [5]:
import itertools
from typing import Literal

rr.init("sequence data")
set_pose_annotation_context()
rr.notebook_show(width=1000, height=700, blueprint=blueprint)

num_frames: int = 50

exo_data: ExoData
for idx, exo_data in enumerate(itertools.islice(sequence, num_frames)):
    rr.set_time("frame idx",sequence=idx)
    for hand_enum in HandSide:
        hand_side: Literal["left", "right"] = hand_enum.name.lower()
        hand_idx: int = hand_enum.value
        
        xyz_gt: Float32[ndarray, "21 3"] = exo_data.xyz[hand_idx]

        rr.log(
            hand_side,
            rr.Points3D(
                xyz_gt,
                colors=(0, 255, 0),
                class_ids=0,
                keypoint_ids=MEDIAPIPE_IDS,
                show_labels=False,
            ),
        )

    for cam_param, bgr in zip(exo_data.cam_params_list, exo_data.bgr_list, strict=True):
        cam_log_path: Path = parent_log_path / cam_param.name   
        image_log_path: Path = cam_log_path / "pinhole" / "video"
        # camera parameters don't change, so we can log them once
        if idx == 0:
            log_pinhole(
                        camera=cam_param,
                        cam_log_path=cam_log_path,
                        image_plane_distance=0.1,
                        static=True
                    )
        rr.log(
            f"{image_log_path}",
            rr.Image(
                bgr,
                color_model=rr.ColorModel.BGR
            ).compress(jpeg_quality=15),
        )
        for hand_enum in HandSide:
            hand_side: Literal["left", "right"] = hand_enum.name.lower() # Get "left" or "right"
            hand_idx: int = hand_enum.value

            xyz_gt: Float32[ndarray, "21 3"] = exo_data.xyz[hand_idx]
            # np.nan is used to indicate that a joint is not confidently detected
            uv: Float32[ndarray, "21 2"] = exo_data.uv_dict[cam_param.name][hand_idx]
            # np.nan is used to indicate that a joint is not confidently detected
            uv[uv == -1] = np.nan

            # find min/max to generate a bounding box, add a small padding
            uv_min:Float32[ndarray, "2"] = np.nanmin(uv, axis=0)
            uv_max:Float32[ndarray, "2"] = np.nanmax(uv, axis=0)

            xyxy:Float32[ndarray, "4"] = np.concatenate([uv_min, uv_max], axis=0)
            # if nan skip logging keypoints and bboxes
            if np.any(np.isnan(xyxy)):
                continue

            # keypoints
            rr.log(
                f"{image_log_path}/{hand_side}_keypoints",
                rr.Points2D(
                    uv,
                    colors=(0, 255, 0),
                    class_ids=0,
                    keypoint_ids=MEDIAPIPE_IDS,
                    show_labels=False,
                ),
            )
            # bounding boxes
            rr.log(
                    f"{image_log_path}/{hand_side}_xyxy",
                    rr.Boxes2D(
                        array=xyxy,
                        array_format=rr.Box2DFormat.XYXY,
                        labels=[hand_side],
                    )
            )

Viewer()

If not, consider setting `RERUN_NOTEBOOK_ASSET`. Consult https://pypi.org/project/rerun-notebook/0.23.1/ for details.



## 1. RGB Frames -> 2D Keypoints

In [6]:
import rerun.blueprint as rrb
from rtmlib import Wholebody

device = 'cuda'  # cpu, cuda, mps
backend = 'onnxruntime'  # opencv, onnxruntime, openvino

wholebody = Wholebody(to_openpose=False,
                      mode='performance',  # 'performance', 'lightweight', 'balanced'. Default: 'balanced'
                      backend=backend, device=device)

bgr: UInt8[ndarray, "H W 3"] = exo_data.bgr_list[-1]
wb_output:tuple[Float[ndarray, "..."], Float[ndarray, "..."]] = wholebody(bgr)

keypoints: Float[ndarray, "n_dets 133 2"] = wb_output[0]
scores: Float[ndarray, "n_dets 133"] = wb_output[1]
uvc: Float[ndarray, "n_dets 133 3"] = np.concatenate(
    [keypoints, scores[..., np.newaxis]], axis=-1
)

LEFT_START_IDX: int = 112
RIGHT_START_IDX: int = 91
left_hand: Float[ndarray, "n_dets 21 3"] = uvc[:, LEFT_START_IDX:LEFT_START_IDX+21, :]
right_hand: Float[ndarray, "n_dets 21 3"] = uvc[:, RIGHT_START_IDX:RIGHT_START_IDX+21, :]


minimal_blueprint = rrb.Blueprint(collapse_panels=True)

rr.init("2D keypoints")
set_pose_annotation_context()
rr.notebook_show(width=1000, height=700, blueprint=minimal_blueprint)

rr.log(
    "image",
    rr.Image(
        bgr,
        color_model=rr.ColorModel.BGR
    ).compress(jpeg_quality=15),
)

rr.log(
    "image/2d_keypoints",
    rr.Points2D(
        uvc[0, :, :2],
        colors=(0, 255, 0),
        class_ids=2,
        keypoint_ids=COCO_133_IDS,
        show_labels=False,
    ),
)

rr.log(
    "image/left_hand_keypoints",
    rr.Points2D(
        left_hand[0, :, :2],
        colors=(255, 0, 0),
        class_ids=0,
        keypoint_ids=MEDIAPIPE_IDS,
        show_labels=False,
    ),
)

rr.log(
    "image/right_hand_keypoints",
    rr.Points2D(
        right_hand[0, :, :2],
        colors=(255, 0, 0),
        class_ids=0,
        keypoint_ids=MEDIAPIPE_IDS,
        show_labels=False,
    ),
)

load /home/pablo/.cache/rtmlib/hub/checkpoints/yolox_m_8xb8-300e_humanart-c2c7a14a.onnx with onnxruntime backend
load /home/pablo/.cache/rtmlib/hub/checkpoints/rtmw-dw-x-l_simcc-cocktail14_270e-384x288_20231122.onnx with onnxruntime backend


2025-05-02 14:16:12.009910146 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-02 14:16:12.009924173 [W:onnxruntime:, session_state.cc:1170 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2025-05-02 14:16:12.092018376 [W:onnxruntime:, graph.cc:4285 CleanUnusedInitializersAndNodeArgs] Removing initializer '1701'. It is not used by any node and should be removed from the model.
2025-05-02 14:16:12.092040427 [W:onnxruntime:, graph.cc:4285 CleanUnusedInitializersAndNodeArgs] Removing initializer '1706'. It is not used by any node and should be removed from the model.
2025-05-02 14:16:12.092050196 [W:onnxruntime:, graph.cc:4285 CleanUnusedInitializersAndNodeArgs] Removing initializer '1709'. It is not us

Viewer()

If not, consider setting `RERUN_NOTEBOOK_ASSET`. Consult https://pypi.org/project/rerun-notebook/0.23.1/ for details.



In [7]:
rr.init("2D keypoints")
set_pose_annotation_context()
rr.notebook_show(width=1000, height=700, blueprint=minimal_blueprint)

for idx, exo_data in enumerate(itertools.islice(sequence, 150)):
    rr.set_time("frame idx",sequence=idx)
    bgr = exo_data.bgr_list[-1]

    wb_output:tuple[Float[ndarray, "..."], Float[ndarray, "..."]] = wholebody(bgr)

    keypoints: Float[ndarray, "n_dets 133 2"] = wb_output[0]
    scores: Float[ndarray, "n_dets 133"] = wb_output[1]
    uvc: Float[ndarray, "n_dets 133 3"] = np.concatenate(
        [keypoints, scores[..., np.newaxis]], axis=-1
    )

    left_hand: Float[ndarray, "n_dets 21 3"] = uvc[:, LEFT_START_IDX:LEFT_START_IDX+21, :]
    right_hand: Float[ndarray, "n_dets 21 3"] = uvc[:, RIGHT_START_IDX:RIGHT_START_IDX+21, :]


    rr.log(
        "image",
        rr.Image(
            bgr,
            color_model=rr.ColorModel.BGR
        ).compress(jpeg_quality=15),
    )

    rr.log(
        "image/left_hand_keypoints",
        rr.Points2D(
            left_hand[0, :, :2],
            colors=(255, 0, 0),
            class_ids=0,
            keypoint_ids=MEDIAPIPE_IDS,
            show_labels=False,
        ),
    )

    rr.log(
        "image/right_hand_keypoints",
        rr.Points2D(
            right_hand[0, :, :2],
            colors=(0, 255, 0),
            class_ids=0,
            keypoint_ids=MEDIAPIPE_IDS,
            show_labels=False,
        ),
    )

Viewer()

If not, consider setting `RERUN_NOTEBOOK_ASSET`. Consult https://pypi.org/project/rerun-notebook/0.23.1/ for details.



## 2. 3D triangulated keypoints on the first frame

For the sake of simplicity, we'll use the provided 3D keypoints form the dataset

In [8]:
# Get the ExoData for the selected frames
exo_data: ExoData = sequence[0]
xyz_gt: Float[ndarray, "2 21 3"] = exo_data.xyz
xyz_template_left: Float[ndarray, "21 3"] = xyz_gt[0]
xyz_template_right: Float[ndarray, "21 3"] = xyz_gt[1]

# log triangulated points
rr.init("3D Triangulated")
set_pose_annotation_context()
rr.notebook_show(width=1000, height=700, blueprint=minimal_blueprint)

for hand_enum in HandSide:
    hand_side: Literal["left", "right"] = hand_enum.name.lower()
    hand_idx: int = hand_enum.value
    rr.log(
        f"image/gt_{hand_side}",
        rr.Points3D(
            xyz_gt[hand_idx],
            colors=(0, 255, 0),
            class_ids=0,
            keypoint_ids=MEDIAPIPE_IDS,
            show_labels=False,
        ),
    )

Viewer()

If not, consider setting `RERUN_NOTEBOOK_ASSET`. Consult https://pypi.org/project/rerun-notebook/0.23.1/ for details.



## 3. Kinematic Skeleton

With initial xyz keypoints, we'll use these to set the joint lengths in the kinematic skeleton

In [9]:
from jax import jit

from pi0_lerobot.mano.mano_jax import JointsOnly, mp_to_mano

# --- Knematic Skeleton Setup ---
# JIT compile the MANO function for performance
xyz_left_gt: Float[Array, "21 3"] = jnp.array(xyz_template_left[mp_to_mano, :])
left_joints = JointsOnly(template_joints=xyz_left_gt)
left_joints_jit = jit(left_joints)

In [10]:
import logging

import ipywidgets as widgets
from IPython.display import HTML, display
from rerun.notebook import Viewer

display(HTML("""
<style>
/* slider label text */
.widget-label {
    color: #ffffff !important;
}

/* numeric readout on the right side of sliders */
.widget-readout {
    color: #ffffff !important;
}

/* optional: make the thumb pop on dark backgrounds */
.jupyter-widgets input[type=range]::-webkit-slider-thumb {
    border: 2px solid #ffffff;
}
</style>
"""))

# Set the logging level for the root logger to ERROR to suppress WARNING messages
logging.getLogger().setLevel(logging.ERROR)

# --- Rerun Initialization ---
rr.init("mano_sliders_example")
viewer = Viewer(height=600, width=800)


rr.set_time("time", sequence=0)
set_pose_annotation_context()
rr.send_blueprint(minimal_blueprint)

# --- MANO Joint Name Mapping ---
# Derived from MEDIAPIPE_ID2NAME and mp_to_mano provided by user
# This maps the MANO *pose parameter* joint index (1-15) to its name
MANO_POSE_JOINT_ID2NAME = {
    1: "INDEX_MCP",  # MANO index 1 (mp 5)
    2: "INDEX_PIP",  # MANO index 2 (mp 6)
    3: "INDEX_DIP",  # MANO index 3 (mp 7)
    4: "MIDDLE_MCP", # MANO index 4 (mp 9)
    5: "MIDDLE_PIP", # MANO index 5 (mp 10)
    6: "MIDDLE_DIP", # MANO index 6 (mp 11)
    7: "PINKY_MCP",  # MANO index 7 (mp 17)
    8: "PINKY_PIP",  # MANO index 8 (mp 18)
    9: "PINKY_DIP",  # MANO index 9 (mp 19)
    10: "RING_MCP",  # MANO index 10 (mp 13)
    11: "RING_PIP",  # MANO index 11 (mp 14)
    12: "RING_DIP",  # MANO index 12 (mp 15)
    13: "THUMB_MCP", # MANO index 13 (mp 1) - Note: MP calls this CMC
    14: "THUMB_PIP", # MANO index 14 (mp 2) - Note: MP calls this MCP
    15: "THUMB_IP",  # MANO index 15 (mp 3) - Note: MP calls this IP
}

# --- Slider Creation (48 sliders) ---

sliders = []
for i in range(48):
    # First 3 are global rotation, rest are joint rotations
    if i < 3:
        description = f'Global Rot Axis {i}'
    else:
        joint_idx = (i - 3) // 3 + 1
        axis_idx = (i - 3) % 3
        joint_name = MANO_POSE_JOINT_ID2NAME.get(joint_idx, f'Joint {joint_idx}') # Use name or fallback
        description = f'{joint_name} Axis {axis_idx}'
    
    sliders.append(widgets.FloatSlider(
            min=-3.14, 
            max=3.14, 
            step=0.01, 
            value=0.0, 
            description=description,
            continuous_update=True, # Only update when slider released
            layout=widgets.Layout(width='200px'), # Reduced width
        ))

# --- Update Function ---
def update_mano(change=None):
    # Gather slider values
    pose_coeffs_list = [s.value for s in sliders]
    pose_coeffs = jnp.array(pose_coeffs_list).reshape(1, 48)
    
    # Define default translation (can be made interactive too if needed)
    trans = jnp.zeros((1, 1, 3))
    
    # Run MANO forward kinematics
    keypoints = left_joints_jit(pose_coeffs, trans) # Use JIT compiled version
    keypoints_np: Float[ndarray, "21 3"] = np.array(keypoints[0])  # Extract the first batch
    
    # Log to Rerun (convert JAX array to NumPy for Rerun)
    rr.log("world/mano_joints",
           rr.Points3D(
               positions=keypoints_np,
               colors=(0, 255, 0),
               class_ids=0,
               keypoint_ids=MEDIAPIPE_IDS,
               show_labels=False, )) # Adjusted radius

# --- Observe Sliders ---
for slider in sliders:
    slider.observe(update_mano, names='value')

# --- Layout ---
# Arrange sliders in a grid (e.g., 16 rows, 3 columns)
slider_grid = widgets.GridBox(sliders, layout=widgets.Layout(
    grid_template_columns='repeat(3, 220px)', # Reduced column width
    grid_gap='2px 5px' # Reduced gap between sliders
))

# Capture viewer display
viewer_output = widgets.Output()
with viewer_output:
    display(viewer)

# Combine viewer and sliders (viewer on left, sliders on right)
ui_layout = widgets.HBox([viewer_output, slider_grid])

# --- Initial Log & Display ---
update_mano() # Log the initial pose (all zeros)
display(ui_layout)

## Example Optimization on a single timestep using multiview 2d keypoints

In [11]:
from jaxopt._src.levenberg_marquardt import LevenbergMarquardtState

from pi0_lerobot.mano.mano_optimization_jax import (
    JointAndScaleOptimization,
    LossWeights,
    OptimizationResults,
    proj_3d_vectorized,
)

loss_weights = LossWeights(
    keypoint_2d=0.1,
    depth=0.0,
    temp=0.0,
)

exo_data_list: list[ExoData] = [sequence[50]]

P_list: list[Float[ndarray, "3 4"]] = [exo_cam.projection_matrix for exo_cam in exo_data_list[0].cam_params_list]
Pall: Float[ndarray, "n_views 3 4"] = np.stack(P_list, axis=0)

new_joint_and_scale_optimizer = JointAndScaleOptimization(
    xyz_template=np.stack([xyz_template_left, xyz_template_right], axis=0),
    Pall=Pall,
    loss_weights=loss_weights,
    num_iters=30)

rr.init("Optimization over sequence data")
set_pose_annotation_context()
rr.notebook_show(width=1200, height=700, blueprint=blueprint)

for cam_param in exo_data.cam_params_list:
    cam_log_path: Path = parent_log_path / cam_param.name   
    image_log_path: Path = cam_log_path / "pinhole" / "video"
    # camera parameters don't change, so we can log them once
    log_pinhole(
                camera=cam_param,
                cam_log_path=cam_log_path,
                image_plane_distance=0.1,
                static=True
            )

exo_data: ExoData
for idx, exo_data in enumerate(sequence):
    rr.set_time("frame idx",sequence=idx)

    uv_batch_left: None
    uv_batch_right: None
    for hand_enum in HandSide:
        hand_side: Literal["left", "right"] = hand_enum.name.lower()
        hand_idx: int = hand_enum.value

        xyz_gt: Float32[ndarray, "21 3"] = exo_data.xyz[hand_idx]

        # create a batch of xyz points
        xyz_gt_hom: Float[ndarray, "21 4"] = np.concatenate([xyz_gt, np.ones_like(xyz_gt)[..., 0:1]], axis=-1)
        xyz_gt_hom: Float[Array, "1 21 4"] = jnp.array(xyz_gt_hom)[jnp.newaxis, ...]

        # create a batch of uv points in vectorized form
        uv_batch:Float[Array, "1 n_views 21 2"] = proj_3d_vectorized(
            xyz_hom=xyz_gt_hom,
            P=jnp.array(Pall)
        )
        # remove the n_frames dimension and convert to numpy
        uv_batch = np.array(uv_batch).squeeze(axis=0)
        match hand_side:
            case "left":
                uv_batch_left = uv_batch
            case "right":
                uv_batch_right = uv_batch
            case _:
                raise ValueError(f"Invalid hand side: {hand_side}")
            
        rr.log(
            f"{hand_side}",
            rr.Points3D(
                xyz_gt,
                colors=(0, 255, 0),
                class_ids=0,
                keypoint_ids=sequence.hand_ids,
                show_labels=False,
            ),
        )

    optim_out: tuple[OptimizationResults, LevenbergMarquardtState] = new_joint_and_scale_optimizer(
        uv_left_pred_batch=uv_batch_left,
        uv_right_pred_batch=uv_batch_right
    )
    uv_batch_gt:Float[ndarray, "n_views 2 21 2"] = np.stack([uv_batch_left, uv_batch_right], axis=1)
    optimized_result: OptimizationResults = optim_out[0]

    for hand_enum in HandSide:
        hand_side: Literal["left", "right"] = hand_enum.name.lower()
        hand_idx: int = hand_enum.value

        match hand_side:
            case "left":
                xyz_optim: Float[ndarray, "21 3"] = optimized_result.xyz_mano[0]
            case "right":
                xyz_optim = optimized_result.xyz_mano[1]
            case _:
                raise ValueError(f"Invalid hand side: {hand_side}")
        
        # project optimized points to 2D
        xyz_optim_hom: Float[ndarray, "21 4"] = np.concatenate([xyz_optim, np.ones_like(xyz_optim)[..., 0:1]], axis=-1)
        xyz_optim_hom: Float[Array, "1 21 4"] = jnp.array(xyz_optim_hom)[jnp.newaxis, ...]
        uv_optim_batch: Float[Array, "1 n_views 21 2"] = proj_3d_vectorized(
            xyz_hom=jnp.array(xyz_optim_hom),
            P=jnp.array(Pall)
        )
        # remove the n_frames dimension
        uv_optim_batch: Float[ndarray, "n_views 21 2"] = np.array(uv_optim_batch).squeeze(axis=0)

        rr.log(
            f"{hand_side}_optim",
            rr.Points3D(
                xyz_optim,
                colors=(0, 255, 0),
                class_ids=1,
                keypoint_ids=sequence.hand_ids,
                show_labels=False,
            ),
        )
        # log the optimized 2D points
        for cam_param, uv_optim, uv_gt in zip(exo_data.cam_params_list, uv_optim_batch, uv_batch_gt, strict=True):
            cam_log_path: Path = parent_log_path / cam_param.name   
            image_log_path: Path = cam_log_path / "pinhole" / "video"

            rr.log(
                f"{image_log_path}/{hand_side}_keypoints_optim",
                rr.Points2D(
                    uv_optim,
                    colors=(0, 255, 0),
                    class_ids=1,
                    keypoint_ids=MEDIAPIPE_IDS,
                    show_labels=False,
                ),
            )

            rr.log(
                f"{image_log_path}/{hand_side}_keypoints_gt",
                rr.Points2D(
                    uv_gt[hand_idx],
                    colors=(0, 255, 0),
                    class_ids=0,
                    keypoint_ids=MEDIAPIPE_IDS,
                    show_labels=False,
                ),
            )


/home/pablo/0Dev/repos/pi0-lerobot/.pixi/envs/default/lib/python3.11/site-packages/jaxopt/_src/levenberg_marquardt.py:506: UserWarning: The linear solver cholesky that requires materialization of J^T.J matrix is used with materialize_jac=False, which may cause a computational overhead. Consider using either a matrix-free iterative solver such as cg or bicg or using materialize_jac=True.
  warnings.warn(f"The linear solver {self.solver} that requires materialization of "


Tracing JIT, can take a while...
Trace Done


Viewer()